In [19]:
import trimesh
import numpy as np

mesh = trimesh.load_mesh("./dataset/mesh_simplify/109_0.obj", process=False)
F = mesh.faces  # 3 vertices [faces, 3]
V = mesh.vertices # 3 coordinates [vertices, 3]

Fs = mesh.faces.shape[0]
face_coordinate = V[F.flatten()].reshape(-1, 9) # 3 vertices, 3 coordinates[faces, 9]

face_center = V[F.flatten()].reshape(-1, 3, 3).mean(axis=1) # 3 vertices, 3 coordinates [faces, 3]

vertex_normals = mesh.vertex_normals 
face_normals = mesh.face_normals
face_curvs = np.vstack([
    (vertex_normals[F[:, 0]] * face_normals).sum(axis=1),
    (vertex_normals[F[:, 1]] * face_normals).sum(axis=1),
    (vertex_normals[F[:, 2]] * face_normals).sum(axis=1),
]) # 3 coordinates [faces, 3]
feats = []
if 1:
    feats.append(mesh.area_faces)# [faces, ]
if 1:
    feats.append(face_normals.T) # 3 coordinates [faces, 3]
if 1:
    feats.append(face_center.T) # 3 coordinates [faces, 3]
if 1:
    feats.append(np.sort(mesh.face_angles, axis=1).T) # 3 angles [faces, 3]   
if 1:
    feats.append(np.sort(face_curvs, axis=0)) # 3 coordinates [faces, 3]

feats = np.vstack(feats)
feats = np.array(feats, dtype=np.float32)

patch_num = Fs // 4 // 4 // 4
allindex = np.array(list(range(0, patch_num * 64)))
indices = allindex.reshape(-1, patch_num).transpose(1, 0)

feats_patch = feats[:, indices]
print(feats_patch)
center_patch = face_center[indices]
cordinates_patch = face_coordinate[indices]
faces_patch = mesh.faces[indices]
if patch_num < 256:
    feats_patcha = np.concatenate((feats_patch, np.zeros((10, 256 - patch_num, 64), dtype=np.float32)), 1)
    center_patcha = np.concatenate((center_patch, np.zeros((256 - patch_num, 64, 3), dtype=np.float32)), 0)
    cordinates_patcha = np.concatenate((cordinates_patch, np.zeros((256 - patch_num, 64, 9), dtype=np.float32)), 0)
    faces_patcha = np.concatenate((faces_patch, np.zeros((256 - patch_num, 64, 3), dtype=np.int)), 0)
else: 
    feats_patcha = feats_patch
    center_patcha = center_patch
    cordinates_patcha = cordinates_patch
    faces_patcha = faces_patch

Fs_patcha = np.array(patch_num*64)


[[[ 6.3143969e-01  2.3736801e+00  2.1508548e+00 ...  5.3158426e+00
    7.1477139e-01  1.3471149e+00]
  [ 2.0472240e+00  2.0851388e+00  2.1335847e+00 ...  7.8200688e+00
    1.8407078e+00  1.6356629e+00]
  [ 9.8958898e-01  5.8269233e-01  2.1448817e+00 ...  4.4200897e+00
    2.1087210e+00  7.2736233e-01]
  ...
  [ 2.6348156e-01  2.0895236e+00  2.4802496e+00 ...  6.6487795e-01
    1.2226967e+00  3.5705428e+00]
  [ 2.8034256e+00  2.1004057e+00  1.0359645e+00 ...  1.3618761e+00
    1.8466532e+00  4.0049281e+00]
  [ 2.3358407e+00  2.1461499e+00  3.4467062e-01 ...  9.8189853e-02
    1.3406851e+00  6.1761694e+00]]

 [[-3.1935230e-01 -7.5388539e-01 -9.2881316e-01 ...  7.3029697e-01
    2.4321626e-01  3.7579152e-01]
  [-3.2644773e-01 -7.5247884e-01 -9.3633133e-01 ...  7.2833115e-01
    2.5211608e-01  3.8805205e-01]
  [-3.1509551e-01 -7.3586082e-01 -9.3139982e-01 ...  7.4213958e-01
    2.8052914e-01  4.2237815e-01]
  ...
  [-7.1260124e-01 -9.5604634e-01  6.5242225e-01 ...  2.4320805e-01
    3.3064